<a href="https://colab.research.google.com/github/tb0se/RL-MiniHack-Project/blob/Dev/src/reinforce/REINFORCE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# SETUP NLE(GOOGLE COLAB)

# Python and most build deps
!sudo apt update
!sudo apt-get install -y build-essential autoconf libtool pkg-config python3-dev python3-pip python3-numpy git flex bison libbz2-dev

# recent cmake version
!wget -O - https://apt.kitware.com/keys/kitware-archive-latest.asc 2>/dev/null | sudo apt-key add -
!sudo apt-add-repository 'deb https://apt.kitware.com/ubuntu/ bionic main'
!sudo apt-get update && apt-get --allow-unauthenticated install -y cmake kitware-archive-keyring

!sudo rm $(which cmake)
!$(which cmake) --version

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [1,812 kB]
Get:12 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main amd64 Packages [928 kB]
Ign:13 https://developer.download.nvidia.com/compute

In [ ]:
!pip3 install nle
!pip3 install minihack

     |████████████████████████████████| 6.8 MB 10.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Using cached pybind11-2.8.1-py2.py3-none-any.whl (208 kB)
  Created wheel for nle: filename=nle-0.7.3-cp37-cp37m-linux_x86_64.whl size=2874635 sha256=157974887d5b60f5bdd9cbbcbfae839e4e9e3d6271e78a97a051706db7474a40
  Stored in directory: /root/.cache/pip/wheels/25/9a/cc/5df4c522352fb289d67b08b16cbb28b2131d982798343e681f
Successfully built nle
     |████████████████████████████████| 259 kB 11.7 MB/s 


In [ ]:
import os
import time
import shutil
import numpy as np
import matplotlib.pyplot as plt
import cv2
from collections import deque


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

import gym
from gym import spaces
from gym.wrappers import FrameStack
from nle import nethack
import minihack

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

True

# Gym Wrappers

In [ ]:
class SkipFrame(gym.Wrapper):
    def __init__(self, env, skip):
        """Return only every `skip`-th frame"""
        super().__init__(env)
        self._skip = skip

    def step(self, action):
        """Repeat action, and sum reward"""
        total_reward = 0.0
        done = False
        for i in range(self._skip):
            # Accumulate reward and repeat the same action
            obs, reward, done, info = self.env.step(action)
            total_reward += reward
            if done:
                break
        return obs, total_reward, done, info

class PyTorchFrame(gym.ObservationWrapper):
    """Image shape to num_channels x height x width"""

    def __init__(self, env,obs_type):
        super(PyTorchFrame, self).__init__(env)
        self.obs_type = obs_type
        shape = self.observation_space[obs_type].shape
        self.observation_space = gym.spaces.Box(
            low=0.0, high=1.0, shape=(shape[-1], shape[0], shape[1]), dtype=np.uint8
        )

    def observation(self, observation):
        return np.rollaxis(observation[self.obs_type], 2)

# class FlattenObservation(gym.ObservationWrapper):
#     r"""Observation wrapper that flattens the observation."""

#     def __init__(self, env, obs_type):
#         super(FlattenObservation, self).__init__(env)
#         shape = env.observation_space[obs_type].shape
#         print(shape)
#         self.observation_space = spaces.flatten_space(env.observation_space)
#         print(self.observation_space.shape)

#     def observation(self, observation):
#         return spaces.flatten(self.env.observation_space, observation)

# Simple Neural Network (Policy)

In [ ]:
class PolicyNetwork(nn.Module):
    def __init__(self, obs_space=4, hidden=16, act_space=2, obs_type='glyphs_crop'):
        ''' A simple Neural Network.
        :params obs_space: Observation space (Default=4)
        :params hidden: Hiddens size (Default=16)
        :params act_space: Action space (Default=2)
        '''
        super(PolicyNetwork, self).__init__()
        self.obs_type = obs_type

        # Convolutional and pooling layers
        if obs_type == 'glyphs_crop':
          self.conv1 = nn.Conv2d(obs_space, 32, 5, padding=2)
          self.conv2 = nn.Conv2d(32, 64, 3, padding=2)
          self.conv3 = nn.Conv2d(64, 64, 3, stride=2)
          # Linear layers
          self.linear1 = nn.Linear(64*5*5, hidden)
          self.linear2 = nn.Linear(hidden, act_space)

        elif obs_type == 'pixel_crop':
          self.conv1 = nn.Conv2d(obs_space, 32, 8, stride=4)
          self.conv2 = nn.Conv2d(32, 64, 5, stride=2)
          self.conv3 = nn.Conv2d(64, 64, 3, stride=1)
          self.pool = nn.MaxPool2d(2,2)
          # Linear layers
          self.linear1 = nn.Linear(64*7*7, hidden)
          self.linear2 = nn.Linear(hidden, act_space)
        
        # Dropout
        self.dropout = nn.Dropout(p=0.6)

    def forward(self, x):

      # 3 Conv,relu
      x = F.relu(self.conv1(x)) 
      x = F.relu(self.conv2(x)) 
      x = F.relu(self.conv3(x)) 

      if self.obs_type == 'pixel_crop':
        x = self.pool(x)
      
      # Flatten image
      if self.obs_type == 'pixel_crop':
        x = x.reshape(-1, 64*7*7)
      elif self.obs_type == 'glyphs_crop':
        x = x.view(-1, 64*5*5)

      x = self.linear1(x)
      x = F.relu(x)
      x = self.dropout(x)
      actions = self.linear2(x)

      act_probs = F.softmax(actions, dim=1)

      return act_probs

# REINFORCE Agent

In [ ]:
class Agent():
    
    def __init__(self,obs_size, action_size, policy_model, optimizer, gamma):
        self.eps = np.finfo(np.float32).eps.item()
        self.obs_size = obs_size
        self.action_size = action_size
        self.policy_model = policy_model
        self.optim = optimizer
        self.gamma = gamma
        
        self.states = []
        self.actions = []
        self.gradients = []
        self.rewards = []
        self.probs = []
        
    def __compute_returns(self,rewards):
        returns = []
        cumul_rets = 0
        
        for reward in reversed(rewards):
            cumul_rets = reward + cumul_rets*self.gamma
            returns.insert(0, cumul_rets)

        # Baseline
        returns = torch.tensor(returns).to(DEVICE)
        returns = (returns - returns.mean()) / (returns.std() + self.eps)
        return returns
        
    def choose_action(self,  state):
        state = torch.from_numpy(state).float().to(DEVICE)
        batch_state = state[None, ...]

        with torch.no_grad():
          probs = self.policy_model(batch_state)
        state = state.detach()

        dist = Categorical(probs)
        action = dist.sample()

        # Attempt at stacked states
        # max_vals, max_indices = probs.max(1)
        # max_idx = max_vals.argmax()
        # action = max_indices[max_idx]


        return action.item(), dist.log_prob(action)
    
    def save_trajectory(self, state, action, reward, prob):
        self.states.append(state)
        self.actions.append(action)
        self.rewards.append(reward)
        self.probs.append(prob)
        
    
    def train(self,):
        G = self.__compute_returns(self.rewards)
        
        policy_loss= []
        for ret,prob in zip(G, self.probs):
            policy_loss.append(-prob*ret)
        
        # Backpropagation
        self.optim.zero_grad()
        # policy_loss = torch.cat(policy_loss).sum()
        policy_loss = sum(policy_loss)
        policy_loss = torch.tensor(policy_loss, dtype=torch.float16, requires_grad=True)
        policy_loss.backward()
        self.optim.step()
        
        # Reset
        self.states, self.probs, self.gradients, self.rewards = [],[],[],[]

    def save_checkpoint(self, ep_num):
      chkpts_dir = './model_chkpts'
      if not os.path.isdir(chkpts_dir): 
        os.mkdir(chkpts_dir)
      
      t = time.localtime()
      timestamp = time.strftime('%b-%d-%Y_%H:%M', t)
      f_path = f'{chkpts_dir}/model_{ep_num}_{timestamp}.pt'
      
      torch.save({
            'epoch': ep_num,
            'model_state_dict': self.policy_model.state_dict(),
            'optimizer_state_dict': self.optim.state_dict(),
            }, f_path )
      
      # if is_best:
      #   best_dir = './best_model_chkpt'
      #   if not os.path.isdir(best_dir): 
      #     os.mkdir(best_dir)

      #   best_fpath = f'{best_dir}/best_model.pt'
      #   shutil.copyfile(f_path, best_fpath)

    def load_checkpoint(self,path,is_train=True):
      chckpt = torch.load(path)

      self.policy_model.load_state_dict(chckpt['model_state_dict'])
      self.optim.load_state_dict(chckpt['optimizer_state_dict'])

      curr_ep = chckpt['epoch']

      if is_train:
        self.policy_model.train()
      else:
        self.policy_model.eval()

      return curr_ep

In [ ]:
def main(env, agent, seed, num_episodes, max_episode_len, obs_type,save_freq, start_ep=0, print_freq=100):
    
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  np.random.seed(seed)
  env.seed(seed)
  
  tot_scores = []
  
  for ep in range(start_ep,num_episodes):
    
    state = env.reset()
    score = 0
    
    # Create episode
    for step in range(max_episode_len):
        act, prob = agent.choose_action(np.array(state))
        
        new_state, reward, done, _ = env.step(act)
        
        score += reward
    
        agent.save_trajectory(state, act, reward, prob)
        
        if done:
            break
        state = new_state
    
    tot_scores.append(score)
    
    # For every step in episode
    agent.train()
    
    if ep % print_freq == 0:
        print(f'Episode {ep}\tLast score: {score}\tAverage reward: {np.array(tot_scores).mean()}')
    
    if ep % save_freq == 0:
      print('Saving model checkpoint...')
      agent.save_checkpoint(ep)

# Training

In [ ]:
# Hyperparameters
hyper_params = {
    "seed": 54,
    "env": "MiniHack-Quest-Hard-v0",
    "obs-type":"pixel_crop",
    "learning_rate":1e-2,
    "gamma":0.99,
    "num-episodes":50000,
    "num-steps":int(1e5),
    "print-freq":500,
    "save-freq": 1000,
}


# Environment
env = gym.make(hyper_params['env'], observation_keys=(hyper_params['obs-type'],))
env = SkipFrame(env, skip=4)
env = PyTorchFrame(env,hyper_params['obs-type'])
# env = FrameStack(env, num_stack=4)
print('Action space:',env.action_space.n)
print('Observation shape:',env.observation_space.shape)


# Policy (CNN)
env_obs_space = env.observation_space.shape[0]
policy_model = PolicyNetwork(obs_space=env_obs_space, hidden=128, act_space=env.action_space.n, obs_type=hyper_params['obs-type'])
print(policy_model,'\n')
policy_model.to(DEVICE)
optimizer = optim.Adam(policy_model.parameters(), lr=hyper_params['learning_rate'])

# REINFORCE Agent
agent = Agent(env_obs_space, env.action_space.n,
              policy_model, optimizer, gamma=hyper_params['gamma'])

# Uncomment to load a saved model checkpoint
# path = f'./model_chkpts/model_{ep_num}_{timestamp}.pt'
# start_ep = agent.load_checkpoint(path)

# Train
main(env, agent, seed=hyper_params['seed'],num_episodes=hyper_params['num-episodes'], max_episode_len=hyper_params['num-steps'], 
     obs_type=hyper_params['obs-type'], print_freq=hyper_params['print-freq'],save_freq=hyper_params['save-freq'] )

Action space: 78
Observation shape: (3, 144, 144)
PolicyNetwork(
  (conv1): Conv2d(3, 32, kernel_size=(8, 8), stride=(4, 4))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(2, 2))
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear1): Linear(in_features=3136, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=78, bias=True)
  (dropout): Dropout(p=0.6, inplace=False)
) 



/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:67: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Episode 0	Last score: -9.719999999999969	Average reward: -9.719999999999969
Saving model checkpoint...


/usr/local/lib/python3.7/dist-packages/nle/env/base.py:695: UserWarning: Warning: smooth quitting of game failed, aborting.
  warnings.warn("Warning: smooth quitting of game failed, aborting.")


Episode 500	Last score: -9.789999999999967	Average reward: -9.404231536926119
Episode 1000	Last score: -9.769999999999968	Average reward: -9.45993006993004
Saving model checkpoint...
Episode 1500	Last score: -9.789999999999967	Average reward: -9.473137908061263
Episode 2000	Last score: -9.609999999999973	Average reward: -9.456716641679131
Saving model checkpoint...
Episode 2500	Last score: -8.749999999999991	Average reward: -9.453034786085537
Episode 3000	Last score: -9.659999999999972	Average reward: -9.467637454181911
Saving model checkpoint...
Episode 3500	Last score: -9.879999999999965	Average reward: -9.477386461011111
Episode 4000	Last score: -9.71999999999997	Average reward: -9.465828542864255
Saving model checkpoint...
Episode 4500	Last score: -9.559999999999972	Average reward: -9.466605198844672
Episode 5000	Last score: -9.76999999999997	Average reward: -9.464965006998574
Saving model checkpoint...
Episode 5500	Last score: -9.539999999999973	Average reward: -9.463839301945074


KeyboardInterrupt: ignored

# Plots

# Visualise environment

## 1. Install rendering libraries

In [ ]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

## 2. Import rendering libraries

In [ ]:
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only

import math
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay

## 3. Create virtual display

In [ ]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

## 4. Render function

In [ ]:
def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")

## 5. Gym wrapper for rendering

In [ ]:
cv2.ocl.setUseOpenCL(False)

class RenderRGB(gym.Wrapper):
    def __init__(self, env, key_name="pixel"):
        super().__init__(env)
        self.last_pixels = None
        self.viewer = None
        self.key_name = key_name

        render_modes = env.metadata['render.modes']
        render_modes.append("rgb_array")
        env.metadata['render.modes'] = render_modes

    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        # self.last_pixels = obs[self.key_name]
        self.last_pixels = obs
        return obs, reward, done, info

    def render(self, mode="human", **kwargs):
        img = self.last_pixels
        img = np.swapaxes(img,0,2)
        # print(test.shape)

        # Hacky but works
        if mode != "human":
            return img
        else:
            from gym.envs.classic_control import rendering

            if self.viewer is None:
                self.viewer = rendering.SimpleImageViewer()
            self.viewer.imshow(img)
            return self.viewer.isopen

    def reset(self):
        obs = self.env.reset()
        # self.last_pixels = obs[self.key_name]
        self.last_pixels = obs
        return obs

    def close(self):
        if self.viewer is not None:
            self.viewer.close()
            self.viewer = None

## Visualise

In [ ]:
def visual(env,agent ,num_episodes=50):
  vis_scores = []


  for ep in range(num_episodes):
    state = env.reset()
    score = 0

    while True:
      env.render()

      act, log_prob = agent.choose_action(np.array(state))
      new_state, reward, done, info = env.step(act)
      score += reward
      if done:
        break
      state = new_state
    
    vis_scores.append(score)

  env.close()
  show_video()

  return vis_scores

In [ ]:
path = f'./model_chkpts/model_7000_Nov-10-2021_21:28.pt'
agent = Agent(env_obs_space, env.action_space.n,
              policy_model, optimizer, gamma=hyper_params['gamma'])
start_ep = agent.load_checkpoint(path)

In [ ]:
env = gym.make(hyper_params['env'], observation_keys=(hyper_params['obs-type'],))
env = PyTorchFrame(env,hyper_params['obs-type'])
env = RenderRGB(env, 'pixel_crop')
env = gym.wrappers.Monitor(env, "video", force=True)

scores = visual(env,agent)